The following tutorials are available from the [Wallaroo Tutorials Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/notebooks_in_prod).

# Stage 4: Regular Batch Inference

In Stage 3: Deploy the Model in Wallaroo, the housing model created and tested in Stage 2: Training Process Automation Setup was uploaded to a Wallaroo instance and added to the pipeline `housing-pipe` in the workspace `housepricing`.  This pipeline can be deployed at any point and time and used with new inferences.

For the purposes of this demo, let's say that every month we find the newly entered and still-unsold houses and predict their sale price.

The predictions are entered into a staging table, for further inspection before being joined to the primary housing data table.

We show this as a notebook, but this can also be scripted and scheduled, using CRON or some other process.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `postprocess.py`: Formats the data after inference by the model is complete.
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
  * `wallaroo_client.py`: Additional methods used with the Wallaroo instance to create workspaces, etc.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.

## Steps

This process will use the following steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and the `housepricing` workspace.
* [Deploy the Pipeline](#deploy-the-pipeline): Deploy the pipeline to prepare it to run inferences.
* [Read In New House Listings](#read-in-new-house-listings): Read in the previous month's house listings and submit them to the pipeline for inference.
* [Send Predictions to Results Staging Table](#send-predictions-to-results-staging-table): Add the inference results to the results staging table.

### Connect to Wallaroo

Connect to the Wallaroo instance and set the `housepricing` workspace as the current workspace.

In [1]:
import json
import pickle
import wallaroo
import pandas as pd
import numpy as np
import pyarrow as pa
import datetime

import simdb # module for the purpose of this demo to simulate pulling data from a database

from wallaroo_client import get_workspace

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)


### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

In [3]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [4]:
workspace_name = 'housepricing'
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

In [5]:
new_workspace = get_workspace(workspace_name)
_ = wl.set_current_workspace(new_workspace)

### Deploy the Pipeline

Deploy the `housing-pipe` workspace established in Stage 3: Deploy the Model in Wallaroo (`03_deploy_model.ipynb`).

In [6]:
pipeline = get_pipeline(pipeline_name)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

Waiting for deployment - this will take up to 45s .......... ok


name,housing-pipe
created,2023-10-17 15:54:31.622653+00:00
last_updated,2023-10-17 16:04:58.090379+00:00
deployed,True
tags,
versions,"e2986533-e02c-47f7-9c0e-3fff1107ca1b, 4343d96d-4b5a-42dd-b11a-5c8de5f00922, b203ef3d-9cd5-4cb4-86c8-490f41c18c9b, 13f71061-7bfc-49fe-bca4-58ddfcdaee23, 2c974b7f-f217-4023-b1a7-9be380c2363f, 4f5e218b-8c70-452b-8e0e-6de3b61043a4, 9e2b219f-4d89-4770-bcab-d62b953707e0"
steps,preprocess
published,False


### Read In New House Listings

From the data store, load the previous month's house listing, prepare it as a DataFrame, then submit it for inferencing.

In [7]:
conn = simdb.simulate_db_connection()



# create the query
query = f"select * from {simdb.tablename} where date > DATE(DATE(), '-1 month') AND sale_price is NULL"
print(query)

# read in the data
# can't have null values - turn them into 0
newbatch = pd.read_sql_query(query, conn)
newbatch['sale_price'] = newbatch.sale_price.apply(lambda x: 0)
display(newbatch.shape)
display(newbatch.head(10).loc[:, ["id", "date", "list_price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot"]])

select * from house_listings where date > DATE(DATE(), '-1 month') AND sale_price is NULL


(964, 22)

,id,date,list_price,bedrooms,bathrooms,sqft_living,sqft_lot
0,9215400105,2023-09-18,450000.0,3,1.75,1250,5963
1,1695900060,2023-10-01,535000.0,4,1.00,1610,2982
2,9545240070,2023-09-18,660500.0,4,2.25,2010,9603
3,1432900240,2023-09-28,205000.0,3,1.00,1610,8579
4,1400300055,2023-09-18,425000.0,2,1.00,770,5040
5,7960900060,2023-09-24,2900000.0,4,3.25,5050,20100
6,6378500125,2023-09-21,436000.0,2,1.00,1040,7538
7,2022069200,2023-09-25,455000.0,4,2.50,2210,49375
8,9412900055,2023-09-25,405000.0,3,1.75,2390,6000
9,7424700045,2023-10-03,2050000.0,5,3.00,3830,8480


In [8]:
# query = {'query': newbatch.to_json()}

result = pipeline.infer(newbatch)
# display(result)
predicted_prices = pd.DataFrame(result['out.variable'].apply(lambda x: x[0])).rename(columns={'out.variable':'prediction'})
display(predicted_prices[0:5])

,prediction
0,508255.0
1,500198.0
2,539598.0
3,270739.0
4,346586.0


### Send Predictions to Results Staging Table

Take the predicted prices based on the inference results so they can be joined into the `house_listings` table.

Once complete, undeploy the pipeline to return the resources back to the Kubernetes environment.

In [9]:
result_table = pd.DataFrame({
    'id': newbatch['id'],
    'saleprice_estimate': predicted_prices['prediction']
})

display(result_table)

result_table.to_sql('results_table', conn, index=False, if_exists='append')

,id,saleprice_estimate
0,9215400105,508255.0
1,1695900060,500198.0
2,9545240070,539598.0
3,1432900240,270739.0
4,1400300055,346586.0
...,...,...
959,3304300300,577492.0
960,6453550090,882930.0
961,1760650820,271484.0
962,3345700207,537434.0


In [10]:
# Display the top of the table for confirmation
pd.read_sql_query("select * from results_table limit 5", conn)

,id,saleprice_estimate
0,9215400105,508255.0
1,1695900060,500198.0
2,9545240070,539598.0
3,1432900240,270739.0
4,1400300055,346586.0


In [12]:
conn.close()
pipeline.undeploy()

 ok


name,housing-pipe
created,2023-10-17 15:54:31.622653+00:00
last_updated,2023-10-17 16:04:58.090379+00:00
deployed,False
tags,
versions,"e2986533-e02c-47f7-9c0e-3fff1107ca1b, 4343d96d-4b5a-42dd-b11a-5c8de5f00922, b203ef3d-9cd5-4cb4-86c8-490f41c18c9b, 13f71061-7bfc-49fe-bca4-58ddfcdaee23, 2c974b7f-f217-4023-b1a7-9be380c2363f, 4f5e218b-8c70-452b-8e0e-6de3b61043a4, 9e2b219f-4d89-4770-bcab-d62b953707e0"
steps,preprocess
published,False


From here, organizations can automate this process.  Other features could be used such as data analysis using Wallaroo assays, or other features such as shadow deployments to test champion and challenger models to find which models provide the best results.